In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('all_recipes_data.csv')
df.head()

,name,likes,time,servings,link,calories,fat,carbs,fiber,sugar,...,contains_glutten,fruits,vegetables,grains,dairy_alternatives,herbs_and_spices,sweeteners,oils,condiments_and_sauces,grains_and_cereals
0,Whipped Coffee,0.92,15.0,1,https://tasty.co/recipe/whipped-coffee,132.0,0.0,28.0,0.0,28.0,...,0,0,0,0,0,0,1,0,0,0
1,Ratatouille,0.95,72.0,8,https://tasty.co/recipe/ratatouille,118.0,3.0,22.0,5.0,9.0,...,0,0,1,0,0,1,0,1,0,0
2,Homemade Dutch Oven Bread,0.98,30.0,8,https://tasty.co/recipe/homemade-dutch-oven-bread,259.0,0.0,54.0,1.0,0.0,...,1,0,0,1,0,0,0,1,0,0
3,Spaghetti With Garlic And Oil,0.93,30.0,4,https://tasty.co/recipe/spaghetti-with-garlic-...,234.0,3.0,42.0,1.0,1.0,...,0,0,0,0,0,1,0,1,0,0
4,3-Ingredient Banana Oat Smoothie,0.93,30.0,1,https://tasty.co/recipe/3-ingredient-banana-oa...,552.0,12.0,95.0,11.0,32.0,...,0,1,0,1,0,0,0,0,0,0


#### Normalization of Feature Matrix:

In [3]:
# Normalize feature matrix
normalized_features = (df.drop(['name', 'link'], axis=1) - df.drop(['name', 'link'], axis=1).mean()) / df.drop(['name', 'link'], axis=1).std()


- We begin by extracting the feature matrix from the DataFrame df, excluding the 'name' and 'link' columns. These columns are dropped because they contain non-numeric data and are not used for similarity calculation.
- We then normalize the feature matrix by subtracting the mean of each feature and dividing by the standard deviation. This standardization process ensures that all features are on a similar scale, which is essential for accurate similarity calculation.


#### Calculation of Pairwise Cosine Similarity:

In [4]:
# Calculate pairwise cosine similarity
cosine_sim_matrix = cosine_similarity(normalized_features, normalized_features)


- We use the cosine_similarity function from scikit-learn to calculate pairwise cosine similarity between all pairs of recipes in the normalized feature matrix. The resulting cosine_sim_matrix is a square matrix where each element represents the cosine similarity between two recipes.


#### User Profile Creation:

In [10]:
# Create user profile vector with all zeros except for the favorite categories
user_profile = pd.Series(0, index=normalized_features.columns)
user_profile['fruits', 'vegetables'] = 1  # Example: set favorite categories to 1
user_profile['contains_soy'] = 0 # user allergies
user_profile['time'] = 15


- We create a user profile representing the user's preferences. In this example, the user profile consists of their favorite ingredient categories ('fruits' and 'vegetables') for the first recipe in the DataFrame df.

#### Calculation of Similarity between User Profile and Recipes:

In [11]:
# Calculate similarity between user profile and recipes
user_similarities = cosine_similarity([user_profile.values], normalized_features)

- We calculate the cosine similarity between the user profile and all recipes in the normalized feature matrix, excluding the 'fruits' and 'vegetables' columns. This step generates a similarity score for each recipe indicating how similar it is to the user's preferences.

#### Generation of Top Recommendations:

In [12]:
# Get top recommendations
top_indices = user_similarities.argsort()[0][::-1][:10]  # Get indices of top 10 most similar recipes
top_recipes = df.iloc[top_indices][['name', 'link']]


- We identify the indices of the top 10 most similar recipes based on the similarity scores obtained in the previous step.
- Finally, we retrieve the top recommended recipes from the original DataFrame df based on their indices and extract the 'Recipe' and 'link' columns.

In [13]:
top_recipes

,name,link
235,Gardenscape Focaccia,https://tasty.co/recipe/gardenscape-focaccia
599,Heirloom Tomato Tart With Vegan Basil Ricotta,https://tasty.co/recipe/heirloom-tomato-tart-w...
816,Small Batch Bread And Butter Pickles,https://tasty.co/recipe/small-batch-bread-and-...
656,Coconut Ceviche,https://tasty.co/recipe/coconut-ceviche
413,Easy Low-Flour Cast Iron Focaccia Bread,https://tasty.co/recipe/easy-low-flour-cast-ir...
338,Fresh Corn Salad,https://tasty.co/recipe/fresh-corn-salad
942,Sri Lankan Bibikkan,https://tasty.co/recipe/sri-lankan-bibikkan
621,Mushroom “Bacon”,https://tasty.co/recipe/mushroom-bacon
337,Garden Vegetable Vegan Quiche,https://tasty.co/recipe/garden-vegetable-vegan...
983,Vegan Caesar Salad,https://tasty.co/recipe/vegan-caesar-salad


In [20]:
def recommend(df, user_pref, user_allergies, top_n):
    # Normalize feature matrix
    normalized_features = (df.drop(['name', 'link'], axis=1) - df.drop(['name', 'link'], axis=1).mean()) / df.drop(['name', 'link'], axis=1).std()
    
    # Ensure user preferences and allergies are included in the feature matrix columns
    user_features = user_pref + user_allergies
    missing_features = [feature for feature in user_features if feature not in normalized_features.columns]
    if missing_features:
        raise ValueError(f"The following user preferences/allergies are not found in the feature matrix columns: {missing_features}")

    # Create user profile vector with all zeros except for the favorite categories
    user_profile = pd.Series(0, index=normalized_features.columns)
    
    # Set favorite categories to 1
    for pref in user_pref:
        user_profile[pref] = 1
    
    # Set allergies to 0
    for allergy in user_allergies:
        user_profile[allergy] = 0
    
    # Calculate similarity between user profile and recipes
    user_similarities = cosine_similarity([user_profile.values], normalized_features)
    
    # Get top recommendations
    top_indices = user_similarities.argsort()[0][::-1][:top_n]  # Get indices of top N most similar recipes
    top_recipes = df.iloc[top_indices][['name', 'link']]
    
    return top_recipes


### Example Usage

In [22]:
recommend(df, ['vegetables'], [], 10)

,name,link
962,Leczo,https://tasty.co/recipe/leczo
780,Ackee Scramble,https://tasty.co/recipe/ackee-scramble
558,Healthy White Bean Dip Snack-Prep,https://tasty.co/recipe/healthy-white-bean-dip...
254,Kenyan Masala Chips,https://tasty.co/recipe/kenyan-masala-chips
476,Smoked Paprika Potatoes,https://tasty.co/recipe/smoked-paprika-potatoes
683,Veggie Skins Chips,https://tasty.co/recipe/veggie-skins-chips
299,Vegan Roasted Red Pepper Cream Sauce,https://tasty.co/recipe/vegan-roasted-red-pepp...
991,The Best Ratatouille,https://tasty.co/recipe/the-best-ratatouille
30,One-Pan Breakfast Potatoes,https://tasty.co/recipe/one-pan-breakfast-pota...
350,Freezer Prep Hearty Veggie Soup,https://tasty.co/recipe/freezer-prep-hearty-ve...
